Import Library

In [1]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau  

In [2]:
base_dir = os.path.abspath(os.getcwd()) + os.path.join("\\Data\\Loon-colon-image")

os.listdir(base_dir)

['test', 'train', 'val']

In [3]:
train_dir = os.path.join(base_dir, "train")    

os.listdir(train_dir)

['adenocarcinoma', 'benign', 'squamous_cell_carcinoma']

In [4]:
validation_dir = os.path.join(base_dir, "val")    

os.listdir(validation_dir)

['adenocarcinoma', 'benign', 'squamous_cell_carcinoma']

In [5]:

def random_contrast(image):
    contrast_factor = tf.random.uniform([], 0.5, 1.5)
    image = tf.image.adjust_contrast(image, contrast_factor)
    return image

# Menambahkan fungsi random_contrast pada train_datagen dan val_datagen
train_datagen = ImageDataGenerator(
                preprocessing_function=random_contrast,  # Fungsi kontras acak
                rescale=1./255,
                rotation_range=30,
                horizontal_flip=True,
                shear_range=0.1,
                zoom_range=0.1,
                fill_mode='nearest',
                validation_split=0.2
)

val_datagen = ImageDataGenerator(
                preprocessing_function=random_contrast,  # Fungsi kontras acak
                rescale=1./255,
                validation_split=0.2
)


In [6]:
train_generator = train_datagen.flow_from_directory(
        train_dir,  # direktori data latih
        target_size=(150, 150),  # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=4,
        shuffle=True,
        # karena ini merupakan masalah klasifikasi 2 kelas, gunakan class_mode = 'binary'
        class_mode='categorical')
 
validation_generator = val_datagen.flow_from_directory(
        validation_dir, # direktori data validasi
        target_size=(150, 150), # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=32, # karena ini merupakan masalah klasifikasi 2 kelas gunakan class_mode = 'binary'
        shuffle=False,
        class_mode='categorical')

Found 12000 images belonging to 3 classes.
Found 3000 images belonging to 3 classes.


In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),  # Sesuaikan ukuran Dense layer
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(3, activation='softmax')  # Output layer dengan 3 kelas
])

d:\Python\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 15, 15, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 5, 5, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 2, 2, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │     2,098,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │         1,539 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,535,299 (24.93 MB)

 Trainable params: 6,535,299 (24.93 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # Loss untuk 3 kelas
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_generator,
    epochs=40,                   # jumlah epoch, bisa ditambah sesuai kebutuhan
    validation_data=validation_generator,
    verbose=2,
    callbacks=[early_stopping]
)


Epoch 1/40


d:\Python\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


3000/3000 - 523s - 174ms/step - accuracy: 0.3420 - loss: 1.0961 - val_accuracy: 0.3333 - val_loss: 1.0989
Epoch 2/40
3000/3000 - 264s - 88ms/step - accuracy: 0.3330 - loss: 1.1526 - val_accuracy: 0.3333 - val_loss: 1.0990
Epoch 3/40
3000/3000 - 493s - 164ms/step - accuracy: 0.3340 - loss: 1.0990 - val_accuracy: 0.3333 - val_loss: 1.0994
Epoch 4/40
3000/3000 - 625s - 208ms/step - accuracy: 0.3275 - loss: 1.0990 - val_accuracy: 0.3333 - val_loss: 1.0990
Epoch 5/40
3000/3000 - 673s - 224ms/step - accuracy: 0.3294 - loss: 1.0990 - val_accuracy: 0.3333 - val_loss: 1.0987
Epoch 6/40
3000/3000 - 662s - 221ms/step - accuracy: 0.3320 - loss: 1.0989 - val_accuracy: 0.3333 - val_loss: 1.0986
Epoch 7/40
3000/3000 - 517s - 172ms/step - accuracy: 0.3328 - loss: 1.0989 - val_accuracy: 0.3333 - val_loss: 1.0988
Epoch 8/40
3000/3000 - 310s - 103ms/step - accuracy: 0.3301 - loss: 1.0989 - val_accuracy: 0.3333 - val_loss: 1.0988
Epoch 9/40
3000/3000 - 352s - 117ms/step - accuracy: 0.3299 - loss: 1.0989 -

In [ ]:
train_loss, train_accuracy = model.evaluate(train_generator)

val_loss, val_accuracy = model.evaluate(validation_generator)

print(f"Training Accuracy: {train_accuracy*100:.2f}%")
print(f"Validation Accuracy: {val_accuracy*100:.2f}%")